In [103]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [64]:
# utils
words = open('../files/names.txt','r').read().splitlines()
chars = sorted(list(set([c for w in words for c in w]))+['.'])

ctoi = {c:i for i,c in enumerate(chars)}
stoi = {s:i for i,s in enumerate(f'{c1}{c2}' for c1 in chars for c2 in chars)}

itoc = {i:c for c,i in ctoi.items()}
itos = {i:s for s,i in stoi.items()}


**E01** : train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model? \

Obvious improvement over the bigram model. Again, counts model (true probs) is better than NN. Tried to implement a decaying gradient coefficient to get closer to true minima.

***Counts***

In [95]:
# %%capture
import torch 


def train_counts_model(words):
    # counts isn't really required
    """
    counts = {}
    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            trigram = (f'{c1}{c2}', c3)
            counts[trigram]= counts.get(trigram, 0) + 1

    strs, chars = set([k[0] for k in counts.keys()]), sorted(list(set([k[1] for k in counts.keys()])))
    len(strs), len(chars)
    """

    N = torch.zeros((27*27, 27), dtype=torch.int32) # . doesn't follow .

    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            s = f'{c1}{c2}'
            c = c3
            N[stoi[s], ctoi[c]] += 1
        
    P = (N).float()
    P/=P.sum(1, keepdim=True)

    # Loss
    log_likelihood = 0.0

    n=0
    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            s = f'{c1}{c2}'
            c = c3
            log_likelihood-=torch.log(P[stoi[s], ctoi[c]])
            n+=1

    print(log_likelihood/n)

train_counts_model(words)

"\ncounts = {}\nfor w in words:\n    w = '.'+w+'.'\n    for c1, c2, c3 in zip(w, w[1:], w[2:]):\n        trigram = (f'{c1}{c2}', c3)\n        counts[trigram]= counts.get(trigram, 0) + 1\n\nstrs, chars = set([k[0] for k in counts.keys()]), sorted(list(set([k[1] for k in counts.keys()])))\nlen(strs), len(chars)\n"

tensor(2.0620)


***NN***

In [110]:
import torch 

def train_nn_model(words):
    x,y = [], []

    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            x.append(stoi[f'{c1}{c2}'])
            y.append(ctoi[c3])

    x = torch.tensor(x)
    y = torch.tensor(y)
    xenc = torch.nn.functional.one_hot(x, num_classes=27*27).float()

    W = torch.randn((27*27, 27), requires_grad=True)
    ix = torch.arange(x.nelement())

    # gradient descent
    grad_coeff = 1000
    losses = [1e10]
    for k in range(1000):
        outs = xenc@W
        exp = outs.exp()
        prob = exp/exp.sum(1, keepdim=True)
        loss = -prob[ix, y].log().mean()
        
        if loss>losses[-1]:
            grad_coeff/=2
        losses.append(loss)
        
        W.grad = None
        loss.backward()

        W.data -= 200*W.grad

    print(min(losses))
    
    return W

train_nn_model(words)

tensor(2.0773, grad_fn=<NegBackward0>)


tensor([[-0.2200, -0.9023, -1.0935,  ..., -0.4423,  0.0375, -0.3824],
        [-4.0203,  1.4028,  1.3171,  ..., -0.6353,  1.2233,  1.0939],
        [-1.2030,  4.7914, -1.1684,  ..., -1.3940,  0.8092, -1.2066],
        ...,
        [ 0.5510, -0.1960,  0.0327,  ..., -1.5166, -0.0539,  1.6817],
        [ 3.3111,  3.0777, -1.8135,  ..., -1.1489, -0.9561, -0.5417],
        [ 1.7711,  3.0932, -0.2070,  ..., -0.9813,  2.4313, -1.6206]],
       requires_grad=True)

In [137]:
def get_loss(x, y, W):
    ix = torch.arange(x.nelement())
    x_enc = torch.nn.functional.one_hot(dev_x, num_classes = 27*27).float()
    exp = (x_enc@W).exp()
    prob = exp/exp.sum(1, keepdim=True)
    return -prob[ix, y].log().mean().shape

get_loss(dev_x, dev_y, W)

torch.Size([19700, 729])

**E02**: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [148]:
train, dev, test = torch.utils.data.random_split(words, [0.8, 0.1, 0.1])

W = train_nn_model(train)

def get_loss(x, y, W):
    ix = torch.arange(x.nelement())
    x_enc = torch.nn.functional.one_hot(x, num_classes = 27*27).float()
    exp = (x_enc@W).exp()
    prob = exp/exp.sum(1, keepdim=True)
    return -prob[ix, y].log().mean()

def get_x_y(words):
    x,y = [], []
    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            x.append(stoi[f'{c1}{c2}'])
            y.append(ctoi[c3])
    x = torch.tensor(x)
    y = torch.tensor(y)
    return x,y

dev_x, dev_y = get_x_y(dev)
test_x, test_y = get_x_y(test)

dev_loss = get_loss(dev_x, dev_y, W)
test_loss = get_loss(test_x, test_y, W)


tensor(2.0735, grad_fn=<NegBackward0>)


In [149]:
dev_loss, test_loss

(tensor(2.1105, grad_fn=<NegBackward0>),
 tensor(2.1097, grad_fn=<NegBackward0>))

both dev and test losses are higher than training loss -> overfitting, high variance. 

**E03**: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [156]:
import torch 

def get_loss(x, y, W):
    ix = torch.arange(x.nelement())
    x_enc = torch.nn.functional.one_hot(x, num_classes = 27*27).float()
    exp = (x_enc@W).exp()
    prob = exp/exp.sum(1, keepdim=True)
    return -prob[ix, y].log().mean()

def get_x_y(words):
    x,y = [], []
    for w in words:
        w = '.'+w+'.'
        for c1, c2, c3 in zip(w, w[1:], w[2:]):
            x.append(stoi[f'{c1}{c2}'])
            y.append(ctoi[c3])
    x = torch.tensor(x)
    y = torch.tensor(y)
    return x,y

train, dev, test = torch.utils.data.random_split(words, [0.8, 0.1, 0.1])

train_x, train_y = get_x_y(train)
dev_x, dev_y = get_x_y(dev)

for smooth_strength in torch.arange(0.01, 0.1, 0.01):
    W = torch.randn((27*27, 27), requires_grad=True)

    # gradient descent
    grad_coeff = 1000
    losses = [1e10]
    for k in range(1000):
        loss = get_loss(train_x, train_y, W)
        # regularization
        loss += smooth_strength * (W**2).mean()
        if loss>losses[-1]:
            grad_coeff/=2
        losses.append(loss)
        
        W.grad = None
        loss.backward()

        W.data -= 200*W.grad

    print(f"smooth strength: {smooth_strength}, train loss: {min(losses)}, dev loss: {get_loss(dev_x, dev_y, W)}")


test_x, test_y = get_x_y(test)

test_loss = get_loss(test_x, test_y, W)


smooth strength: 0.009999999776482582, train loss: 2.092672109603882, dev loss: 2.11802077293396
smooth strength: 0.019999999552965164, train loss: 2.107276439666748, dev loss: 2.118516206741333
smooth strength: 0.029999999329447746, train loss: 2.1193363666534424, dev loss: 2.1201188564300537
smooth strength: 0.03999999910593033, train loss: 2.1294806003570557, dev loss: 2.1220180988311768
smooth strength: 0.05000000074505806, train loss: 2.1388325691223145, dev loss: 2.124297618865967
smooth strength: 0.05999999865889549, train loss: 2.147226095199585, dev loss: 2.1268131732940674
smooth strength: 0.07000000029802322, train loss: 2.155240774154663, dev loss: 2.1292643547058105
smooth strength: 0.07999999821186066, train loss: 2.16278076171875, dev loss: 2.131679058074951
smooth strength: 0.09000000357627869, train loss: 2.1700093746185303, dev loss: 2.134233236312866



**E04**: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [160]:
x_enc = torch.nn.functional.one_hot(x, num_classes = 27*27).float()
exp = (x_enc@W).exp()
exp_new = W[x].exp()
torch.equal(exp, exp_new)

True

**E05**: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

- Simpler and more efficient
- It actually handles the overflow issue internally by subracting the max logit value before applying softmax
For ex. 
if model outputs are z=[1000,900,800], the exponentiations are too large \
we deal with this by normalizing the logits by subtracting the maximum logit value before applying softmax \
this works because shifting all logits by a constant does not change the softmax output (it’s invariant to constant shifts). \
no all our logits are <=0.


**E06**: meta-exercise! Think of a fun/interesting exercise and complete it.
1. Does replacing . with different start and end tags change the loss? Why? 
1. What if we had different num of weights instead of (27, 27)
1. Can the NN loss be lower than the counts model on the training set? What does it mean? 